Testing Out Episode Scraping

In [93]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime as dt
import time

In [7]:
NUM_TEAMS = 3
EPISODES = 10 

BUFFER = 1

base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"
# inital team list

r = requests.post(list_url, json = {"teamId":  5696217}) # arbitrary ID, change to leading ID during challenge

rj = r.json()

teams_df = pd.DataFrame(rj['result']['teams'])
teams_df.sort_values('publicLeaderboardRank', inplace = True)
print(teams_df.shape)
teams_df.head(6)

(167, 20)


,id,teamName,competitionId,teamLeaderId,isBenchmark,createdAfterDeadline,medal,dateMedalAwarded,submissionCount,lastSubmissionDate,publicLeaderboardSubmissionId,publicLeaderboardScore,publicLeaderboardRank,privateLeaderboardSubmissionId,privateLeaderboardScore,privateLeaderboardRank,competition,teamMembers,inboundMergeRequests,outboundMergeRequests
160,5696217,SaltyFish,21723,5993643,False,False,0,None,7,"{'seconds': 1603841393, 'nanos': 90000000}",17744755,None,1,None,None,None,None,"[{'id': 574816, 'displayName': 'Niuhe', 'email...",[],[]
141,5653767,WeKick,21723,5918754,False,False,0,None,4,"{'seconds': 1603436140, 'nanos': 13333300}",17632553,None,2,None,None,None,None,"[{'id': 1970652, 'displayName': 'Kaiwen Zhu', ...",[],[]
134,5645993,s_shohei(チーム募集中),21723,2304617,False,False,0,None,32,"{'seconds': 1603799951, 'nanos': 640000000}",17737241,None,3,None,None,None,None,"[{'id': 2304617, 'displayName': 's_shohei', 'e...",[],[]
48,5590149,kangaroo,21723,5860849,False,False,0,None,9,"{'seconds': 1603289251, 'nanos': 796666700}",17570317,None,4,None,None,None,None,"[{'id': 831173, 'displayName': 'zlw21gxy', 'em...",[],[]
143,5655996,zsp1197 zsp,21723,5941019,False,False,0,None,6,"{'seconds': 1603768989, 'nanos': 700000000}",17731892,None,5,None,None,None,None,"[{'id': 5941019, 'displayName': 'zsp1197 zsp',...",[],[]
43,5589631,Kerbol,21723,364943,False,False,0,None,6,"{'seconds': 1603767945, 'nanos': 436666700}",17580824,None,6,None,None,None,None,"[{'id': 364943, 'displayName': 'xy', 'email': ...",[],[]


In [20]:
teams_df[teams_df['publicLeaderboardSubmissionId'] == 17744755]

,id,teamName,competitionId,teamLeaderId,isBenchmark,createdAfterDeadline,medal,dateMedalAwarded,submissionCount,lastSubmissionDate,publicLeaderboardSubmissionId,publicLeaderboardScore,publicLeaderboardRank,privateLeaderboardSubmissionId,privateLeaderboardScore,privateLeaderboardRank,competition,teamMembers,inboundMergeRequests,outboundMergeRequests
160,5696217,SaltyFish,21723,5993643,False,False,0,None,7,"{'seconds': 1603841393, 'nanos': 90000000}",17744755,None,1,None,None,None,None,"[{'id': 574816, 'displayName': 'Niuhe', 'email...",[],[]


In [97]:
def getTeamEpisodes(team_id):
    r = requests.post(list_url, json = {"teamId":  int(team_id)})
    rj = r.json()

    # update teams list
#     global teams_df
#     teams_df_new = pd.DataFrame(rj['result']['teams'])
    
#     if len(teams_df.columns) == len(teams_df_new.columns) and (teams_df.columns == teams_df_new.columns).all():
#         teams_df = pd.concat( (teams_df, teams_df_new.loc[[c for c in teams_df_new.index if c not in teams_df.index]] ) )
#         teams_df.sort_values('publicLeaderboardRank', inplace = True)
#     else:
#         print('teams dataframe did not match')
    
    # make df
    team_episodes = pd.DataFrame(rj['result']['episodes'])
    team_episodes['avg_score'] = -1;
    
    for i in range(len(team_episodes)):
        agents = team_episodes['agents'].loc[i]
        agent_scores = [a['updatedScore'] for a in agents if a['updatedScore'] is not None]
        team_episodes.loc[i, 'submissionId'] = [a['submissionId'] for a in agents if a['submission']['teamId'] == team_id][0]
        team_episodes.loc[i, 'updatedScore'] = [a['updatedScore'] for a in agents if a['submission']['teamId'] == team_id][0]
        
        if len(agent_scores) > 0:
            team_episodes.loc[i, 'avg_score'] = np.mean(agent_scores)

    for sub_id in team_episodes['submissionId'].unique():
        sub_rows = team_episodes[ team_episodes['submissionId'] == sub_id ]
        max_time = max( [r['seconds'] for r in sub_rows['endTime']] )
        final_score = max( [r['updatedScore'] for r_idx, (r_index, r) in enumerate(sub_rows.iterrows())
                                if r['endTime']['seconds'] == max_time] )

        team_episodes.loc[sub_rows.index, 'final_score'] = final_score
        
    team_episodes.sort_values('avg_score', ascending = False, inplace=True)
    return rj, team_episodes


def saveEpisode(epid, rj):
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})
        
    # save replay
    with open('{}.json'.format(epid), 'w') as f:
        f.write(re.json()['result']['replay'])

    # save episode info
    with open('{}_info.json'.format(epid), 'w') as f:
        json.dump([r for r in rj['result']['episodes'] if r['id']==epid][0], f)
        

In [95]:
base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"


def saveEpisode_v2(epid, output_dir='episodes_dl/'):
    print('saveEpisode_v2: {0}'.format(epid))
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})
        
    # save replay
    with open(output_dir + '{}.json'.format(epid), 'w') as f:
        f.write(re.json()['result']['replay'])


def scrape_save_team_top_submit(team_id, submission_id, output_dir='episodes_dl/'):
    
    print('run start')
    start_time = time.time()
    
    
    # make export directory w/ timestamp of runs
    curr_datetime = dt.datetime.now()
    curr_time = curr_datetime.strftime('%d-%m-%Y-%H-%M-%S')
    output_dir = output_dir + curr_time + '/'
    # make output directory if doesnt exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    
    print('team_id: {0}, submission_id: {1}'.format(team_id, submission_id))
    
    config_dict = { 'team_id': team_id, 'submission_id': submission_id} 
    config_dict['timestamp'] = curr_time
    with open(output_dir + 'config.json', 'w') as file:
        file.write(json.dumps(config_dict))
    
    # get teamEpisodes dataframe
    rj, team_episodes_df = getTeamEpisodes(team_id)
    print('Number of total team episodes: {0}'.format(len(team_episodes_df)))
    top_submit_df = team_episodes_df[team_episodes_df['submissionId'] == submission_id].reset_index(drop=True)
    print('Number of total submission episodes: {0}'.format(len(top_submit_df)))
    # iterate through list of episode ids and get json and save each
    for i in range(len(top_submit_df)):
        episode = top_submit_df['id'].iloc[i]
        saveEpisode_v2(episode, output_dir)
        
        
    end_time = round((time.time() - start_time), 2)
    print("complete: --- %s seconds ---" % end_time)

# Test

In [96]:
team_id = 5653767 # WeKick
submission_id = 17632553
output_dir = 'episodes_dl/WeKick/'
scrape_save_team_top_submit(team_id, submission_id, output_dir)

run start
team_id: 5653767, submission_id: 17632553
Number of total team episodes: 276
Number of total submission episodes: 59
saveEpisode_v2: 4079921
saveEpisode_v2: 4076916
complete: --- 6.59 seconds ---


just try to get episodes of top WeKick agent

In [57]:
team_id = 5653767
rj, team_episodes = getTeamEpisodes(team_id)
print(team_episodes.shape)

(271, 13)


In [59]:
team_episodes.head(2)

,id,competitionId,createTime,endTime,replayUrl,adminNotes,state,type,agents,avg_score,submissionId,updatedScore,final_score
218,4037688,21723,"{'seconds': 1603646164, 'nanos': 522673800}","{'seconds': 1603650695, 'nanos': 487706200}",gs://kaggle-episode-replays/4037688.json,None,completed,public,"[{'id': 11772173, 'submissionId': 17530677, 'r...",1425.124800,17530677.0,1412.925182,1406.47356
202,4013233,21723,"{'seconds': 1603488311, 'nanos': 986106300}","{'seconds': 1603492843, 'nanos': 993588800}",gs://kaggle-episode-replays/4013233.json,None,completed,public,"[{'id': 11723261, 'submissionId': 17530677, 'r...",1420.928658,17530677.0,1407.942957,1406.47356


In [60]:
team_episodes.submissionId.value_counts()

17530677.0    100
17632553.0     58
17675719.0     54
17673445.0     51
17530611.0      1
17530548.0      1
17530493.0      1
17530421.0      1
17530364.0      1
17530305.0      1
17530068.0      1
17529831.0      1
Name: submissionId, dtype: int64

We can see from kaggle leaderboard the submission id latest top scoreboard

In [62]:
submission_id = 17632553

In [63]:
team_episodes[team_episodes.submissionId == submission_id].head()

,id,competitionId,createTime,endTime,replayUrl,adminNotes,state,type,agents,avg_score,submissionId,updatedScore,final_score
267,4079921,21723,"{'seconds': 1603853617, 'nanos': 5225100}","{'seconds': 1603858239, 'nanos': 471754700}",gs://kaggle-episode-replays/4079921.json,None,completed,public,"[{'id': 11856616, 'submissionId': 17744755, 'r...",1410.446684,17632553.0,1409.864612,1409.864612
262,4076916,21723,"{'seconds': 1603842590, 'nanos': 819546200}","{'seconds': 1603842955, 'nanos': 246213900}",gs://kaggle-episode-replays/4076916.json,None,completed,public,"[{'id': 11850722, 'submissionId': 17632553, 'r...",1407.175613,17632553.0,1409.838433,1409.864612
213,4031201,21723,"{'seconds': 1603603715, 'nanos': 203515700}","{'seconds': 1603608127, 'nanos': 954421800}",gs://kaggle-episode-replays/4031201.json,None,completed,public,"[{'id': 11759190, 'submissionId': 17632553, 'r...",1404.772518,17632553.0,1383.870452,1409.864612
217,4037373,21723,"{'seconds': 1603644004, 'nanos': 520703500}","{'seconds': 1603648659, 'nanos': 230959500}",gs://kaggle-episode-replays/4037373.json,None,completed,public,"[{'id': 11771540, 'submissionId': 17632553, 'r...",1403.744994,17632553.0,1376.271043,1409.864612
260,4075975,21723,"{'seconds': 1603839171, 'nanos': 984157400}","{'seconds': 1603839540, 'nanos': 171843100}",gs://kaggle-episode-replays/4075975.json,None,completed,public,"[{'id': 11848765, 'submissionId': 17632553, 'r...",1381.092468,17632553.0,1402.325330,1409.864612


In [66]:
team_episodes[team_episodes.submissionId == submission_id].tail()

,id,competitionId,createTime,endTime,replayUrl,adminNotes,state,type,agents,avg_score,submissionId,updatedScore,final_score
80,3947594,21723,"{'seconds': 1603174004, 'nanos': 623136600}","{'seconds': 1603178486, 'nanos': 399299900}",gs://kaggle-episode-replays/3947594.json,None,completed,public,"[{'id': 11591990, 'submissionId': 17374391, 'r...",708.504494,17632553.0,815.646220,1409.864612
84,3947878,21723,"{'seconds': 1603175443, 'nanos': 738421700}","{'seconds': 1603179773, 'nanos': 887899900}",gs://kaggle-episode-replays/3947878.json,None,completed,public,"[{'id': 11592547, 'submissionId': 17487518, 'r...",693.548476,17632553.0,914.418920,1409.864612
85,3947973,21723,"{'seconds': 1603175806, 'nanos': 476382100}","{'seconds': 1603179937, 'nanos': 474376600}",gs://kaggle-episode-replays/3947973.json,None,completed,public,"[{'id': 11592722, 'submissionId': 17417282, 'r...",686.980804,17632553.0,941.896687,1409.864612
79,3947517,21723,"{'seconds': 1603173643, 'nanos': 800338700}","{'seconds': 1603178477, 'nanos': 424390900}",gs://kaggle-episode-replays/3947517.json,None,completed,public,"[{'id': 11591827, 'submissionId': 17467927, 'r...",650.887067,17632553.0,731.559977,1409.864612
78,3946629,21723,"{'seconds': 1603168999, 'nanos': 475190000}","{'seconds': 1603173396, 'nanos': 184578200}",gs://kaggle-episode-replays/3946629.json,None,completed,validation,"[{'id': 11590051, 'submissionId': 17632553, 'r...",600.000000,17632553.0,600.000000,1409.864612


In [74]:
team_episodes[team_episodes.submissionId == submission_id][team_episodes.id == 3947517]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,id,competitionId,createTime,endTime,replayUrl,adminNotes,state,type,agents,avg_score,submissionId,updatedScore,final_score
79,3947517,21723,"{'seconds': 1603173643, 'nanos': 800338700}","{'seconds': 1603178477, 'nanos': 424390900}",gs://kaggle-episode-replays/3947517.json,None,completed,public,"[{'id': 11591827, 'submissionId': 17467927, 'r...",650.887067,17632553.0,731.559977,1409.864612


In [72]:
team_episodes[team_episodes.submissionId == submission_id].tail(5).head(1).agents.iloc[0]

[{'id': 11591990,
  'submissionId': 17374391,
  'reward': -18.0,
  'index': 0,
  'initialScore': 604.372429591994,
  'initialConfidence': 35.0,
  'updatedScore': 601.3627685887371,
  'updatedConfidence': 35.0,
  'state': 'unspecified',
  'submission': {'id': 17374391,
   'title': '',
   'details': '',
   'submittedUserId': 0,
   'dateSubmitted': {'seconds': 1601410484, 'nanos': 403000000},
   'teamId': 5592649,
   'privateScore': None,
   'publicScore': 0.0,
   'isSelected': False,
   'status': 'complete',
   'isAfterDeadline': False,
   'publicScoreAdditionalDetails': '',
   'privateScoreAdditionalDetails': '',
   'processingDetails': '',
   'contentLength': 0,
   'sourceScriptVersionId': 0,
   'clonedFromSubmissionId': 0,
   'dateInvalidated': None,
   'syncRerunId': 0,
   'publicConfidence': 35.0,
   'team': None,
   'sourceKernelRun': None,
   'syncRerunState': None}},
 {'id': 11591993,
  'submissionId': 17632553,
  'reward': 18.0,
  'index': 1,
  'initialScore': 731.5599768769683,

In [65]:
saveEpisode_v2(4079921)